In [1]:
import os
import sys

def detect_platform():
    if os.path.exists('/kaggle/input'):
        return 'kaggle'
    elif 'google.colab' in sys.modules:
        return 'colab'
    else:
        return 'local'

PLATFORM = detect_platform()
print(f"Running on: {PLATFORM}")

Running on: local


In [ ]:
if PLATFORM == 'kaggle':
    # Kaggle: repo must be attached as a dataset or cloned

    os.system('git clone https://github.com/RudraShivm/Skin-Cancer-Detection.git /kaggle/working/skin-cancer-detection')
    REPO_DIR = '/kaggle/working/skin-cancer-detection'

elif PLATFORM == 'colab':
    # Colab: clone from GitHub
    # For private repo, use Colab secrets
    try:
        from google.colab import userdata
        token = userdata.get('GITHUB_TOKEN')
        clone_url = f'https://{token}@github.com/RudraShivm/Skin-Cancer-Detection.git'
    except Exception:
        clone_url = 'https://github.com/RudraShivm/Skin-Cancer-Detection.git'  # public
    
    os.system(f'git clone {clone_url} /content/skin-cancer-detection')
    REPO_DIR = '/content/skin-cancer-detection'

else:  # local
    import rootutils
    REPO_DIR = str(rootutils.find_root(search_from='.', indicator='.project-root'))

print(f"Repo directory: {REPO_DIR}")

# Change to repo directory and install
os.chdir(REPO_DIR)
print(f"Installing dependency packages...")
os.system('pip install -r requirements.txt -q')  # ← install packages
print(f"Installing the current project as an editable Python package...")
os.system('pip install -e . -q')                 # ← make src/ importable

# Add to Python path
sys.path.insert(0, REPO_DIR)

print("✅ Setup complete")

Repo directory: /home/rudrashivm/code/dl/Skin-Cancer-Detection
✅ Setup complete


In [3]:
# PROJECT_ROOT is needed by rootutils in train.py
os.environ['PROJECT_ROOT'] = REPO_DIR

if PLATFORM == 'kaggle':
    # Kaggle secrets (set in Kaggle → Add-ons → Secrets)
    try:
        from kaggle_secrets import UserSecretsClient
        secrets = UserSecretsClient()
        os.environ['WANDB_API_KEY'] = secrets.get_secret('WANDB_API_KEY')
    except Exception:
        pass  # Secrets not available, skip

elif PLATFORM == 'colab':
    # Colab secrets (set in Colab → 🔑 sidebar)
    try:
        from google.colab import userdata
        os.environ['WANDB_API_KEY'] = userdata.get('WANDB_API_KEY')
    except Exception:
        pass

else:  # local
    # Local: load from .env file if it exists
    env_file = os.path.join(REPO_DIR, '.env')
    if os.path.exists(env_file):
        with open(env_file) as f:
            for line in f:
                line = line.strip()
                if line and not line.startswith('#') and '=' in line:
                    key, val = line.split('=', 1)
                    os.environ[key.strip()] = val.strip()

print(f"env::PROJECT_ROOT: {os.environ.get('PROJECT_ROOT')}")

PROJECT_ROOT: /home/rudrashivm/code/dl/Skin-Cancer-Detection


In [ ]:
DATA_DIR = None
CHECKPOINT_DIR = None
OUTPUT_DIR = None

if PLATFORM == 'kaggle':
    # Kaggle mounts competition data at /kaggle/input/
    DATA_DIR = '/kaggle/input/competitions/isic-2024-challenge'
    
    # Outputs go to /kaggle/working/ (this is what gets saved)
    CHECKPOINT_DIR = '/kaggle/working/checkpoints'
    OUTPUT_DIR = '/kaggle/working/outputs'

elif PLATFORM == 'colab':
    # Option A: Data in Google Drive
    from google.colab import drive
    drive.mount('/content/drive', force_remount=False)
    DATA_DIR = '/content/drive/MyDrive/isic-2024-challenge'
    
    # Option B: Data downloaded to Colab's ephemeral disk
    # DATA_DIR = '/content/isic-2024-challenge'
    
    CHECKPOINT_DIR = f'{REPO_DIR}/checkpoints'
    OUTPUT_DIR = f'{REPO_DIR}/outputs'

else:  # local
    DATA_DIR = os.path.join(REPO_DIR, 'data', 'isic-2024-challenge')
    CHECKPOINT_DIR = os.path.join(REPO_DIR, 'checkpoints')
    OUTPUT_DIR = os.path.join(REPO_DIR, 'outputs')

# Create dirs if needed
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"Data:        {DATA_DIR}")
print(f"Checkpoints: {CHECKPOINT_DIR}")
print(f"Outputs:     {OUTPUT_DIR}")

# Verify data exists
if os.path.exists(DATA_DIR):
    print(f"✅ Data found: {os.listdir(DATA_DIR)[:5]}")
else:
    print(f"⚠️  Data NOT found at {DATA_DIR}")

Data:        /home/rudrashivm/code/dl/Skin-Cancer-Detection/data/isic-2024-challenge
Checkpoints: /home/rudrashivm/code/dl/Skin-Cancer-Detection/checkpoints
Outputs:     /home/rudrashivm/code/dl/Skin-Cancer-Detection/outputs
✅ Data found: ['sample_submission.csv', 'test-image.hdf5', 'test-metadata.csv', 'train-image.hdf5', 'train-image']


In [ ]:
# Suppress warnings for neat output
import warnings, os

warnings.filterwarnings('ignore')
os.environ['PYTHONWARNINGS'] = 'ignore'
os.environ['LIGHTNING_DISABLE_UPGRADE_WARNINGS'] = '1'
os.environ['LITMODULE_DISABLE_UPGRADE_WARNINGS'] = '1'

# Build Hydra overrides from the paths we detected above
hydra_overrides = [
    f"data.data_dir={DATA_DIR}",
    "experiment=isic_efficientnet_b0",
    "trainer.accelerator=gpu",
    "extras.print_config=false",   # ← removes the big CONFIG tree
    "extras.enforce_tags=false",   # ← removes the tags warning
    "logger=wandb",
]

# Platform-specific overrides
if PLATFORM == 'kaggle':
    hydra_overrides += [
        "trainer.max_epochs=20",
        "trainer.precision=32",    # GPU available
        "data.num_workers=4",
        "~callbacks.rich_progress_bar",
        "~callbacks.model_summary",
        "data.data_fraction=0.1"
    ]
elif PLATFORM == 'collab':
    hydra_overrides += [
        "trainer.max_epochs=20",
        "trainer.precision=32",    # GPU available
        "data.num_workers=4",
        "data.data_fraction=0.1"
    ]
else:  # local
    hydra_overrides += [
        "debug=default",
    ]

# Build command
override_str = ' '.join(hydra_overrides)
cmd = f'python src/train.py {override_str}'

print(f"Running:\n  {cmd}\n")
os.system(cmd)

Running:
  python src/train.py data.data_dir=/home/rudrashivm/code/dl/Skin-Cancer-Detection/data/isic-2024-challenge experiment=isic_efficientnet_b0 debug=default

[2026-02-17 20:43:32,234][src.utils.utils][INFO] - [rank: 0] Printing config tree with Rich! <cfg.extras.print_config=True>
CONFIG
├── data
│   └── _target_: src.data.isic_datamodule.ISICDataModule                       
│       data_dir: /home/rudrashivm/code/dl/Skin-Cancer-Detection/data/isic-2024-
│       hdf5_file: train-image.hdf5                                             
│       metadata_file: train-metadata.csv                                       
│       batch_size: 32                                                          
│       num_workers: 0                                                          
│       pin_memory: false                                                       
│       persistent_workers: false                                               
│       fold: 0                                 

Seed set to 42


[2026-02-17 20:43:32,495][h5py._conv][DEBUG] - Creating converter from 7 to 5
[2026-02-17 20:43:32,495][h5py._conv][DEBUG] - Creating converter from 5 to 7
[2026-02-17 20:43:32,495][h5py._conv][DEBUG] - Creating converter from 7 to 5
[2026-02-17 20:43:32,495][h5py._conv][DEBUG] - Creating converter from 5 to 7
[2026-02-17 20:43:33,421][__main__][INFO] - [rank: 0] Instantiating model <src.models.isic_module.ISICLitModule>


/home/rudrashivm/.conda/envs/skin-cancer-detection/lib/python3.11/site-packages/timm/models/_factory.py:138: UserWarning: Mapping deprecated model name tf_efficientnet_b0_ns to current tf_efficientnet_b0.ns_jft_in1k.
  model = create_fn(


[2026-02-17 20:43:34,441][timm.models._builder][INFO] - Loading pretrained weights from Hugging Face hub (timm/tf_efficientnet_b0.ns_jft_in1k)
[2026-02-17 20:43:34,448][httpcore.connection][DEBUG] - connect_tcp.started host='huggingface.co' port=443 local_address=None timeout=10 socket_options=None
[2026-02-17 20:43:34,536][httpcore.connection][DEBUG] - connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x7f36ecc2ef90>
[2026-02-17 20:43:34,536][httpcore.connection][DEBUG] - start_tls.started ssl_context=<ssl.SSLContext object at 0x7f36702d2450> server_hostname='huggingface.co' timeout=10
[2026-02-17 20:43:34,559][httpcore.connection][DEBUG] - start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x7f36ecd651d0>
[2026-02-17 20:43:34,560][httpcore.http11][DEBUG] - send_request_headers.started request=<Request [b'HEAD']>
[2026-02-17 20:43:34,560][httpcore.http11][DEBUG] - send_request_headers.complete
[2026-02-17 20:43:34,560][httpcore.

/home/rudrashivm/.conda/envs/skin-cancer-detection/lib/python3.11/site-packages/lightning/fabric/connector.py:571: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
/home/rudrashivm/.conda/envs/skin-cancer-detection/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/accelerator_connector.py:479: You passed `Trainer(accelerator='cpu', precision='16-mixed')` but AMP with fp16 is not supported on CPU. Using `precision='bf16-mixed'` instead.
Using bfloat16 Automatic Mixed Precision (AMP)
You have turned on `Trainer(detect_anomaly=True)`. This will significantly slow down compute speed and is recommended only for model debugging.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
/home/rudrashivm/.conda/envs/skin-cancer-detection/lib/python3.11/site-packages/lightning/pytorch/trainer/setup.py:175: GPU available but not used. You can set it by doing `Trainer(accelerator=

┏━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name           ┃ Type              ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ model          │ EfficientNet      │  4.0 M │ train │     0 │
│ 1 │ criterion      │ BCEWithLogitsLoss │      0 │ train │     0 │
│ 2 │ train_auroc    │ BinaryAUROC       │      0 │ train │     0 ��
│ 3 │ val_auroc      │ BinaryAUROC       │      0 │ train │     0 │
│ 4 │ test_auroc     │ BinaryAUROC       │      0 │ train │     0 │
│ 5 │ train_loss     │ MeanMetric        │      0 │ train │     0 │
│ 6 │ val_loss       │ MeanMetric        │      0 │ train │     0 │
│ 7 │ test_loss      │ MeanMetric        │      0 │ train │     0 │
│ 8 │ val_auroc_best │ MaxMetric         │      0 │ train │     0 │
└───┴────────────────┴───────────────────┴────────┴───────┴───────┘
Trainable params: 4.0 M                                                         
Non-trainable params: 0           


Detected KeyboardInterrupt, attempting graceful shutdown ...


Epoch 0/0  ━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/9731 0:00:24 • 1 day, 6:28:54 0.09it/s 
[2026-02-17 20:44:06,450][src.utils.utils][INFO] - [rank: 0] Output dir: /home/rudrashivm/code/dl/Skin-Cancer-Detection/logs
[2026-02-17 20:44:06,464][httpcore.connection][DEBUG] - close.started
[2026-02-17 20:44:06,479][httpcore.connection][DEBUG] - close.complete


2

In [ ]:
if PLATFORM == 'kaggle':
    # Kaggle auto-saves /kaggle/working/ when notebook finishes
    # Just print where to find them
    print(f"Checkpoints saved to: {CHECKPOINT_DIR}")
    print("Kaggle will persist these when you save the notebook.")
    
    # Optional: commit results back to GitHub
    push_to_github = False 
    if push_to_github:
        from kaggle_secrets import UserSecretsClient
        secrets = UserSecretsClient()
        token    = secrets.get_secret('GITHUB_TOKEN')
        email    = secrets.get_secret('GIT_EMAIL')
        name     = secrets.get_secret('GIT_NAME')
        
        os.system(f'git config user.email "{email}"')
        os.system(f'git config user.name  "{name}"')
        os.system(f'git remote set-url origin https://{token}@github.com/YOU/REPO.git')
        
        os.system('git add outputs/ checkpoints/')
        os.system('git commit -m "Training results from Kaggle"')
        os.system('git push')
        print("✅ Pushed to GitHub")
    else:
        print(f"Results in /kaggle/working/ (auto-saved by Kaggle)")

elif PLATFORM == 'colab':
    # Option A: Save to Google Drive (persists across sessions)
    save_to_drive = True
    if save_to_drive:
        import shutil
        drive_save_dir = '/content/drive/MyDrive/isic-results'
        os.makedirs(drive_save_dir, exist_ok=True)
        shutil.copytree(CHECKPOINT_DIR, f'{drive_save_dir}/checkpoints', dirs_exist_ok=True)
        print(f"✅ Saved to Drive: {drive_save_dir}")
    
    # Option B: Download to local machine
    # from google.colab import files
    # files.download(f'{CHECKPOINT_DIR}/best.ckpt')

else:  # local
    # Already on local machine, nothing special needed
    print(f"Results in: {OUTPUT_DIR}")
    print(f"Checkpoints in: {CHECKPOINT_DIR}")